In [13]:
from web3 import Web3
import os
import json
from pathlib import Path


abi_dir = "./cafecosmos-contracts/abi"

def load_abis(abi_root_dir) -> dict:
    """
    Load all ABI files from the given directory structure.
    Returns a dictionary with contract names as keys and their ABIs as values.
    """
    abis = {}
    root_path = Path(abi_root_dir)
    
    # Walk through all subdirectories
    for contract_dir in root_path.iterdir():
        if contract_dir.is_dir():
            # Look for the .abi.json file in each contract directory
            abi_file = next(contract_dir.glob("*.abi.json"), None)
            if abi_file:
                try:
                    with open(abi_file, 'r') as f:
                        contract_name = contract_dir.name.replace(".sol", "")
                        abis[contract_name] = json.load(f)
                except json.JSONDecodeError as e:
                    print(f"Error loading {abi_file}: {e}")
                except Exception as e:
                    print(f"Error processing {abi_file}: {e}")
    
    return abis


In [14]:
load_abis(abi_dir)

{'LandTablesAndChairsContract': [{'type': 'function',
   'name': 'checkPlaceTableOrChair',
   'inputs': [{'name': 'landId', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'itemId', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'x', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'y', 'type': 'uint256', 'internalType': 'uint256'}],
   'outputs': [],
   'stateMutability': 'nonpayable'},
  {'type': 'function',
   'name': 'checkRemoveTableOrChair',
   'inputs': [{'name': 'landId', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'itemId', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'x', 'type': 'uint256', 'internalType': 'uint256'},
    {'name': 'y', 'type': 'uint256', 'internalType': 'uint256'}],
   'outputs': [],
   'stateMutability': 'nonpayable'},
  {'type': 'event',
   'name': 'Store_SpliceDynamicData',
   'inputs': [{'name': 'tableId',
     'type': 'bytes32',
     'indexed': True,
     'internalType': 'ResourceId'},
  

In [47]:
from web3 import Web3
import os
import json
from pathlib import Path


abi_dir = "./cafecosmos-contracts/abi"

def load_abis(abi_root_dir) -> dict:
    """
    Load all ABI files from the given directory structure.
    Returns a dictionary with contract names as keys and their ABIs as values.
    """
    abis = {}
    root_path = Path(abi_root_dir)
    
    # Walk through all subdirectories
    for contract_dir in root_path.iterdir():
        if contract_dir.is_dir():
            # Look for the .abi.json file in each contract directory
            abi_file = next(contract_dir.glob("*.abi.json"), None)
            if abi_file:
                try:
                    with open(abi_file, 'r') as f:
                        contract_name = contract_dir.name.replace(".sol", "")
                        abis[contract_name] = json.load(f)
                except json.JSONDecodeError as e:
                    print(f"Error loading {abi_file}: {e}")
                except Exception as e:
                    print(f"Error processing {abi_file}: {e}")
    
    return abis


In [54]:
abis = load_abis(abi_dir)
world_abi = abis["IWorld"]
rpc = "https://rpc.garnetchain.com"
world_address = "0x0209FE2b6C1a2B8cE12D8d9D6cf2D78a3Ff324FE"

In [58]:
def extract_errors_from_contracts(contracts):
    all_errors = {}
    for contract_name, abi in contracts.items():
        for item in abi:
            if item['type'] == 'error':
                all_errors[item['name']] = {
                    'contract': contract_name,
                    'selector': None,
                    'inputs': {inp['name']: inp['type'] for inp in item.get('inputs', [])}
                }
    return all_errors

# Extract errors from the contracts
errors_dict = extract_errors_from_contracts(abis)

# Compute selectors for each error
for error_name, error_details in errors_dict.items():
    signature = f"{error_name}({','.join(error_details['inputs'].values())})"
    selector = Web3.keccak(text=signature).hex()[:10]
    errors_dict[error_name]['selector'] = selector

# Output the result
for error_name, error_info in errors_dict.items():
    print(f"Error: {error_name}")
    print(f"  Contract: {error_info['contract']}")
    print(f"  Selector: {error_info['selector']}")
    print(f"  Inputs: {error_info['inputs']}\n")

Error: EncodedLengths_InvalidLength
  Contract: LandQuestTaskProgressUpdate
  Selector: 7149a3c199
  Inputs: {'length': 'uint256'}

Error: Slice_OutOfBounds
  Contract: LandQuestTaskProgressUpdate
  Selector: 23230fa3a3
  Inputs: {'data': 'bytes', 'start': 'uint256', 'end': 'uint256'}

Error: Store_IndexOutOfBounds
  Contract: LandQuestTaskProgressUpdate
  Selector: 7e8578d3de
  Inputs: {'length': 'uint256', 'accessedIndex': 'uint256'}

Error: Store_InvalidResourceType
  Contract: LandQuestTaskProgressUpdate
  Selector: 31b46683f0
  Inputs: {'expected': 'bytes2', 'resourceId': 'bytes32', 'resourceIdString': 'string'}

Error: Store_InvalidSplice
  Contract: LandQuestTaskProgressUpdate
  Selector: a65010b44a
  Inputs: {'startWithinField': 'uint40', 'deleteCount': 'uint40', 'fieldLength': 'uint40'}

Error: FieldLayout_Empty
  Contract: IWorld
  Selector: a019e4dac3
  Inputs: {}

Error: FieldLayout_InvalidStaticDataLength
  Contract: IWorld
  Selector: eba964de36
  Inputs: {'staticDataLeng

In [35]:
w3 = Web3(Web3.HTTPProvider(rpc))

In [37]:
worldw3 = w3.eth.contract(address=world_address, abi=world_abi)

In [38]:
world = worldw3.functions

In [44]:
world.calculateLandCost(1, 1).call()

100000000000

In [62]:
world.placeItem(1, 1, 1, 1).call()

ContractCustomError: ('0xd2838c64000000000000000000000000e919cc7bea733bf4cf3c60e85305b4eff00593260000000000000000000000000000000000000000000000000000000000000000', '0xd2838c64000000000000000000000000e919cc7bea733bf4cf3c60e85305b4eff00593260000000000000000000000000000000000000000000000000000000000000000')

In [128]:
from web3 import Web3
import os
import json
from pathlib import Path


abi_dir = "./cafecosmos-contracts/abi"

def load_abis(abi_root_dir) -> dict:
    """
    Load all ABI files from the given directory structure.
    Returns a dictionary with contract names as keys and their ABIs as values.
    """
    abis = {}
    root_path = Path(abi_root_dir)
    
    # Walk through all subdirectories
    for contract_dir in root_path.iterdir():
        if contract_dir.is_dir():
            # Look for the .abi.json file in each contract directory
            abi_file = next(contract_dir.glob("*.abi.json"), None)
            if abi_file:
                try:
                    with open(abi_file, 'r') as f:
                        contract_name = contract_dir.name.replace(".sol", "")
                        abis[contract_name] = json.load(f)
                except json.JSONDecodeError as e:
                    print(f"Error loading {abi_file}: {e}")
                except Exception as e:
                    print(f"Error processing {abi_file}: {e}")
    
    return abis

def extract_errors_from_contracts(contracts):
    all_errors = {}
    for contract_name, abi in contracts.items():
        for item in abi:
            if item['type'] == 'error':
                all_errors[item['name']] = {
                    'contract': contract_name,
                    'selector': None,
                    'inputs': {inp['name']: inp['type'] for inp in item.get('inputs', [])}
                }
    return all_errors

# Extract errors from the contracts
errors_dict = extract_errors_from_contracts(abis)

# Compute selectors for each error
for error_name, error_details in errors_dict.items():
    signature = f"{error_name}({','.join(error_details['inputs'].values())})"
    selector = Web3.keccak(text=signature).hex()[:10]
    errors_dict[error_name]['selector'] = selector

# Output the result
for error_name, error_info in errors_dict.items():
    print(f"Error: {error_name}")
    print(f"  Contract: {error_info['contract']}")
    print(f"  Selector: {error_info['selector']}")
    print(f"  Inputs: {error_info['inputs']}\n")

class World:
    def __init__(self, rpc, world_address, abis_dir):
        # Initialize Web3
        self.w3 = Web3(Web3.HTTPProvider(rpc))
        
        # Load ABIs (assuming load_abis is a function you have implemented)
        abis = load_abis(abis_dir)
        
        # Initialize the contract
        world_contract = self.w3.eth.contract(address=world_address, abi=abis["IWorld"])
        
        # Extract errors (assuming extract_errors_from_contracts is a function you have implemented)
        self.errors = extract_errors_from_contracts(abis)
        
        # Dynamically add all contract functions to the instance
        for func_name in dir(world_contract.functions):
            # Skip internal/dunder attributes
            if not func_name.startswith("_"):
                # Wrap the function to default to `.call()`
                original_function = getattr(world_contract.functions, func_name)
                setattr(self, func_name, self._wrap_function(original_function))

    def _wrap_function(self, contract_function):
        """Wraps a contract function to default to .call() when invoked."""
        def wrapped_function(*args, **kwargs):
            # Default to .call() and forward any arguments
            return contract_function(*args, **kwargs).call()
        return wrapped_function

Error: EncodedLengths_InvalidLength
  Contract: LandQuestTaskProgressUpdate
  Selector: 7149a3c199
  Inputs: {'length': 'uint256'}

Error: Slice_OutOfBounds
  Contract: LandQuestTaskProgressUpdate
  Selector: 23230fa3a3
  Inputs: {'data': 'bytes', 'start': 'uint256', 'end': 'uint256'}

Error: Store_IndexOutOfBounds
  Contract: LandQuestTaskProgressUpdate
  Selector: 7e8578d3de
  Inputs: {'length': 'uint256', 'accessedIndex': 'uint256'}

Error: Store_InvalidResourceType
  Contract: LandQuestTaskProgressUpdate
  Selector: 31b46683f0
  Inputs: {'expected': 'bytes2', 'resourceId': 'bytes32', 'resourceIdString': 'string'}

Error: Store_InvalidSplice
  Contract: LandQuestTaskProgressUpdate
  Selector: a65010b44a
  Inputs: {'startWithinField': 'uint40', 'deleteCount': 'uint40', 'fieldLength': 'uint40'}

Error: FieldLayout_Empty
  Contract: IWorld
  Selector: a019e4dac3
  Inputs: {}

Error: FieldLayout_InvalidStaticDataLength
  Contract: IWorld
  Selector: eba964de36
  Inputs: {'staticDataLeng

In [129]:
world = World(rpc, world_address, abi_dir)

In [130]:
world.placeItem(1, 1, 1, 1)

ContractCustomError: ('0xd2838c64000000000000000000000000e919cc7bea733bf4cf3c60e85305b4eff00593260000000000000000000000000000000000000000000000000000000000000000', '0xd2838c64000000000000000000000000e919cc7bea733bf4cf3c60e85305b4eff00593260000000000000000000000000000000000000000000000000000000000000000')